In [2]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Resultados do Active Learning

In [3]:
df1 = pd.read_csv("raw_data.csv", index_col=0)
df1 = df1[df1['y'].notnull()]
df1.shape

(307, 5)

In [4]:
df2 = pd.read_csv("active_labels1_done.csv", index_col=0)
df2 = df2[df2['y'].notnull()]

df2.shape

(242, 6)

In [5]:
df_model3 = pd.concat((df2.drop('p',axis=1), df1))
df_model3.to_csv("df_model3.csv")

In [6]:
df2['novo'] = 1

In [7]:
df2.head(1)

,title,upload_date,view_count,tempo_desde_pub,y,p,novo
318,Mindset /w Killa Keim (From DownTown) | BBOY.O...,2021-03-11,4728,201.0,0,463.0,1


In [8]:
from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['y'],df2['p']), roc_auc_score(df2['y'],df2['p'])

(0.45307729683588427, 0.5720455024225828)

In [9]:
df = pd.concat([df1, df2.drop("p", axis=1)])

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [11]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['novo'] = df['novo'].fillna(0)

In [12]:
df_limpo.novo.value_counts()

0.0    307
1.0    242
Name: novo, dtype: int64

## 1. Limpeza da data

In [13]:

df_limpo['date'] = pd.to_datetime(df['upload_date'])

## 2. Limpeza de Views

In [14]:
df_limpo['views'] = df['view_count']

## 3. Features

In [15]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [16]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [17]:
features.head()

,views,views_por_dia
0,735081,181.232988
1,658147,172.109571
2,102757,1317.397436
3,6,-0.008969
4,3134585,2999.602871


# Aumenta validação

In [18]:
base = "2021-09-27"
mask_train = (df_limpo['date'] < base) & (df_limpo['novo'] == 0)

mask_val = (df_limpo['date'] >= base)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((248, 2), (102, 2), (248,), (102,))

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [64]:
title_bow_train.shape

(248, 227)

In [65]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [66]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((248, 229), (102, 229))

In [67]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [68]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [69]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [70]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.3889371006469682, 0.7363636363636363)

ANTES  
ap 0.484250700280112 auc 0.8244897959183674 - mindf=2  


# Aumenta treino

In [90]:
# mask_train = (df_limpo['date'] < base) & (df_limpo['novo'] == 0)
mask_val = (df_limpo['date'] >= base)
# mask_val = (df_limpo['date'] >= base) & (df_limpo['novo'] == 0)
mask_val = (df_limpo['date'] >= base)

In [91]:
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
print(Xtrain.shape, Xval.shape, ytrain.shape, yval.shape)

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])


mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p), roc_auc_score(yval, p)

(248, 2) (102, 2) (248,) (102,)


(0.3889371006469682, 0.7363636363636363)

ANTES
ap 0.484250700280112 auc 0.8244897959183674 - mindf=2


Valid nova, treino antigo  
(0.47753925284809556, 0.7030082778114274

Valid antiga, treino novo  
(0.37737734487734487, 0.7693877551020408)

Tudo novo  
(0.3889371006469682, 0.7363636363636363)